In [1]:
# Импорт основных библиотек
import numpy as np
import pandas as pd
#Импортирование библиотек, загрузка файла
import pandas as pd              #Импорт библиотеки Pandas
import numpy as np               #Импорт библиотеки Numpy
import seaborn as sns            #Импорт библиотеки Seaborn
import matplotlib.pyplot as plt  #Импорт библиотеки Matplotlib



# Импорт библиотек построения графиков и диаграмм
from matplotlib import pyplot as plt
import seaborn as sns

# Указание режима отображения диаграмм
%matplotlib inline

# Импорт основных библиотек машинного обучения
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.preprocessing import MinMaxScaler

# Импорт библиотек моделей машинного обучения
from sklearn.linear_model import LogisticRegression


from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBClassifier
from xgboost import XGBRegressor
from xgboost import XGBClassifier
from lightgbm import LGBMRegressor
from lightgbm import LGBMClassifier

# Импорт библиотек отбора признаков
from sklearn.feature_selection import RFE

# Импорт библиотек оценки моделей машинного обучения
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

In [2]:
train_data = pd.read_parquet("/home/jupyter/datasphere/project/train_dataset_hackaton2023_train.gzip");

In [3]:
train_data['date_diff_post'] = train_data['date_diff_post'].fillna(0)

In [4]:
# Основные статистические характеристики обучающего набора данных
train_data.describe()

,customer_id,date_diff_post,buy_post,revenue,ownareaall_sqm
count,1.212938e+07,1.212938e+07,1.212938e+07,1.212938e+07,1.212938e+07
mean,1.980954e+07,1.631231e+01,7.964846e-01,1.067995e+02,2.155130e+02
std,1.138904e+07,1.592896e+01,4.026126e-01,1.074385e+02,1.225907e+02
min,2.989100e+04,0.000000e+00,0.000000e+00,1.000000e-02,0.000000e+00
25%,1.010852e+07,2.000000e+00,1.000000e+00,4.499000e+01,1.049000e+02
50%,2.127459e+07,1.200000e+01,1.000000e+00,7.998000e+01,2.104000e+02
75%,2.803856e+07,2.600000e+01,1.000000e+00,1.317725e+02,3.000000e+02
max,4.666180e+07,6.000000e+01,1.000000e+00,1.449971e+04,9.787000e+02


In [5]:
# Проверка типов данных и пропущенных значений в обучающем наборе данных
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12129384 entries, 0 to 24261978
Data columns (total 9 columns):
 #   Column          Dtype         
---  ------          -----         
 0   customer_id     int64         
 1   date_diff_post  float64       
 2   buy_post        int64         
 3   group_name      object        
 4   revenue         float64       
 5   startdatetime   datetime64[ns]
 6   dish_name       object        
 7   ownareaall_sqm  float64       
 8   format_name     object        
dtypes: datetime64[ns](1), float64(3), int64(2), object(3)
memory usage: 925.4+ MB


In [6]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()

train_data['dish_name'] = train_data['dish_name'].astype(str)
train_data['ownareaall_sqm'] = train_data['ownareaall_sqm'].astype(str)
train_data['format_name'] = train_data['format_name'].astype(str)

train_data['dish_name'] = labelencoder.fit_transform(train_data['dish_name'])
train_data['ownareaall_sqm'] = labelencoder.fit_transform(train_data['ownareaall_sqm'])
train_data['format_name'] = labelencoder.fit_transform(train_data['format_name'])

train_data['startdatetime'] = (train_data['startdatetime'] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')
train_data.drop('group_name', axis=1, inplace=True)

train_data

,customer_id,date_diff_post,buy_post,revenue,startdatetime,dish_name,ownareaall_sqm,format_name
0,29891,9.0,1,69.99,1670241838,589,363,2
1,29891,9.0,1,190.00,1670241838,982,363,2
2,29891,9.0,1,9.99,1670241838,900,363,2
3,29891,9.0,1,119.99,1670241838,1035,363,2
4,29891,9.0,1,119.99,1670250515,625,363,2
...,...,...,...,...,...,...,...,...
24261974,46661804,0.0,0,7.99,1690913096,1021,363,8
24261975,46661804,0.0,0,84.98,1690913096,501,363,8
24261976,46661804,0.0,0,70.03,1690913096,719,363,8
24261977,46661804,0.0,0,19.00,1690913096,916,363,8


In [ ]:
aggregation_functions = {
    'revenue': lambda x: x.median(),  # медиана
    'startdatetime': 'max',  # Последняя дата
    'dish_name': lambda x: x.mode().iloc[0],  # Наиболее часто встречающеес блюд, которые покупал пользователь
    'ownareaall_sqm': lambda x: x.mode().iloc[0],  # Наиболее часто встречающеес площадей ресторана
    'format_name': lambda x: x.mode().iloc[0]    # Наиболее часто встречающееся значение площади ресторана
}

train_data = train_data.groupby(['customer_id','buy_post', 'date_diff_post']).agg(aggregation_functions).reset_index()
train_data

In [9]:
# Разбивка данных на обучающий и тестовый наборы
# модуль buy_post
columns_to_drop = ['date_diff_post', 'buy_post','customer_id']
X = train_data.drop(columns_to_drop, axis=1)  # Признаки, кроме выручки
y = train_data['buy_post']  # Целевая переменная (выручка)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Создание и обучение модели градиентного бустинга (XGBoost)
xgb_model = XGBClassifier()
xgb_model.fit(X_train, y_train)
xgb_pred = xgb_model.predict(X_test)

# Создание и обучение модели градиентного бустинга (LightGBM)
lgbm_model = LGBMClassifier()
lgbm_model.fit(X_train, y_train)
lgbm_pred = lgbm_model.predict(X_test)

# Создание и обучение модели случайного леса
rf_model = RandomForestClassifier()
rf_model.fit(X_train, y_train)
rf_pred = rf_model.predict(X_test)

# Оценка среднеквадратичной ошибки для каждой модели
xgb_mse = mean_squared_error(y_test, xgb_pred)
lgbm_mse = mean_squared_error(y_test, lgbm_pred)
rf_mse = mean_squared_error(y_test, rf_pred)

# Вывод результатов
print(f"XGBoost MSE: {xgb_mse}")
print(f"LightGBM MSE: {lgbm_mse}")
print(f"Random Forest MSE: {rf_mse}")

# Выбор лучшей модели на основе среднеквадратичной ошибки
best_model = min([(xgb_mse, "XGBoost"), (lgbm_mse, "LightGBM"), (rf_mse, "Random Forest")])[1]
print(f"Лучшая модель для определения оттока: {best_model}")

ValueError: DataFrame.dtypes for data must be int, float, bool or category. When categorical type is supplied, The experimental DMatrix parameter`enable_categorical` must be set to `True`.  Invalid columns:dish_name: object, ownareaall_sqm: object

In [ ]:
train_data

In [ ]:
from sklearn.metrics import f1_score

y_train = (y_train > 0.5).astype(int)
y_test = (y_test > 0.5).astype(int)

xgb_f1_score = f1_score(y_test, xgb_pred)
lgbm_f1_score = f1_score(y_test, lgbm_pred)
rf_f1_score = f1_score(y_test, rf_pred)

# Вывод результатов F1-скора
print(f"XGBoost F1-score: {xgb_f1_score}")
print(f"LightGBM F1-score: {lgbm_f1_score}")
print(f"Random Forest F1-score: {rf_f1_score}")

In [ ]:
# ROC и AUC графики
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, roc_auc_score

# Получение вероятностей для получения ROC и AUC
xgb_proba = xgb_model.predict_proba(X_test)[:, 1]
lgbm_proba = lgbm_model.predict_proba(X_test)[:, 1]
rf_proba = rf_model.predict_proba(X_test)[:, 1]

# Вычисление ROC и AUC
xgb_fpr, xgb_tpr, _ = roc_curve(y_test, xgb_proba)
lgbm_fpr, lgbm_tpr, _ = roc_curve(y_test, lgbm_proba)
rf_fpr, rf_tpr, _ = roc_curve(y_test, rf_proba)

xgb_auc = roc_auc_score(y_test, xgb_proba)
lgbm_auc = roc_auc_score(y_test, lgbm_proba)
rf_auc = roc_auc_score(y_test, rf_proba)

# Построение ROC графика
plt.figure(figsize=(8, 6))
plt.plot(xgb_fpr, xgb_tpr, label=f'XGBoost (AUC = {xgb_auc:.2f})')
plt.plot(lgbm_fpr, lgbm_tpr, label=f'LightGBM (AUC = {lgbm_auc:.2f})')
plt.plot(rf_fpr, rf_tpr, label=f'Random Forest (AUC = {rf_auc:.2f})')
plt.plot([0, 1], [0, 1], linestyle='--', color='grey')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend()
plt.show()

In [ ]:
# Разбивка данных на обучающий и тестовый наборы
# модуль date_diff_post

columns_to_drop = ['date_diff_post','customer_id']
X_Regressor = train_data.drop(columns_to_drop, axis=1)  # Признаки, кроме выручки
y_Regressor = train_data['date_diff_post']  # Целевая переменная (выручка)
X_train_Regressor, X_test_Regressor, y_train_Regressor, y_test_Regressor = train_test_split(X_Regressor, y_Regressor, test_size=0.2, random_state=42)

# Создание и обучение модели градиентного бустинга (XGBoost)
xgb_model_Regressor = XGBRegressor()
xgb_model_Regressor.fit(X_train_Regressor, y_train_Regressor)
xgb_pred_Regressor = xgb_model_Regressor.predict(X_test_Regressor)

# Создание и обучение модели градиентного бустинга (LightGBM)
lgbm_model_Regressor = LGBMRegressor()
lgbm_model_Regressor.fit(X_train_Regressor, y_train_Regressor)
lgbm_pred_Regressor = lgbm_model_Regressor.predict(X_test_Regressor)

# Создание и обучение модели случайного леса
rf_model_Regressor = RandomForestRegressor()
rf_model_Regressor.fit(X_train_Regressor, y_train_Regressor)
rf_pred_Regressor = rf_model_Regressor.predict(X_test_Regressor)

# Оценка среднеквадратичной ошибки для каждой модели
xgb_mse_Regressor = mean_squared_error(y_test_Regressor, xgb_pred_Regressor)
lgbm_mse_Regressor = mean_squared_error(y_test_Regressor, lgbm_pred_Regressor)
rf_mse_Regressor = mean_squared_error(y_test_Regressor, rf_pred_Regressor)

# Вывод результатов
print(f"XGBoost MSE: {xgb_mse_Regressor}")
print(f"LightGBM MSE: {lgbm_mse_Regressor}")
print(f"Random Forest MSE: {rf_mse_Regressor}")

# Выбор лучшей модели на основе среднеквадратичной ошибки
best_model_Regressor = min([(xgb_mse_Regressor, "XGBoost"), (lgbm_mse_Regressor, "LightGBM"), (rf_mse_Regressor, "Random Forest")])[1]
print(f"Лучшая модель для определения date_diff_post: {best_model_Regressor}")

In [ ]:
from sklearn.metrics import mean_absolute_error
xgb_mae_Regressor = mean_absolute_error(y_test_Regressor, xgb_pred_Regressor)
lgbm_mae_Regressor = mean_absolute_error(y_test_Regressor, lgbm_pred_Regressor)
rf_mae_Regressor = mean_absolute_error(y_test_Regressor, rf_pred_Regressor)

# Вывод результатов MAE
print(f"XGBoost MAE: {xgb_mae_Regressor}")
print(f"LightGBM MAE: {lgbm_mae_Regressor}")
print(f"Random Forest MAE: {rf_mae_Regressor}")

best_model_Regressor = min([(xgb_mse_Regressor, "XGBoost"), (lgbm_mse_Regressor, "LightGBM"), (rf_mse_Regressor, "Random Forest")])[1]
print(f"Лучшая модель для определения date_diff_post по MAE: {best_model_Regressor}")

In [ ]:
from sklearn.metrics import r2_score

xgb_r2_Regressor = r2_score(y_test_Regressor, xgb_pred_Regressor)
lgbm_r2_Regressor = r2_score(y_test_Regressor, lgbm_pred_Regressor)
rf_r2_Regressor = r2_score(y_test_Regressor, rf_pred_Regressor)

print(f"XGBoost R²: {xgb_r2_Regressor}")
print(f"LightGBM R²: {lgbm_r2_Regressor}")
print(f"Random Forest R²: {rf_r2_Regressor}")

best_model_Regressor = min([(xgb_mse_Regressor, "XGBoost"), (lgbm_mse_Regressor, "LightGBM"), (rf_mse_Regressor, "Random Forest")])[1]
print(f"Лучшая модель для определения date_diff_post по R²: {best_model_Regressor}")

In [ ]:
from sklearn.metrics import mean_squared_error
import numpy as np

# Рассчет RMSE для каждой модели
xgb_rmse_Regressor = np.sqrt(mean_squared_error(y_test_Regressor, xgb_pred_Regressor))
lgbm_rmse_Regressor = np.sqrt(mean_squared_error(y_test_Regressor, lgbm_pred_Regressor))
rf_rmse_Regressor = np.sqrt(mean_squared_error(y_test_Regressor, rf_pred_Regressor))

# Вывод результатов
print(f"XGBoost RMSE: {xgb_rmse_Regressor}")
print(f"LightGBM RMSE: {lgbm_rmse_Regressor}")
print(f"Random Forest RMSE: {rf_rmse_Regressor}")

# Выбор лучшей модели на основе RMSE
best_model_rmse_Regressor = min([(xgb_rmse_Regressor, "XGBoost"), (lgbm_rmse_Regressor, "LightGBM"), (rf_rmse_Regressor, "Random Forest")])[1]
print(f"Лучшая модель для определения date_diff_post: {best_model_rmse_Regressor}")

In [ ]:
#генерация фалйов

test_data['dish_name'] = test_data['dish_name'].astype(str)
test_data['ownareaall_sqm'] = test_data['ownareaall_sqm'].astype(str)
test_data['format_name'] = test_data['format_name'].astype(str)

test_data['dish_name'] = labelencoder.fit_transform(test_data['dish_name'])
test_data['ownareaall_sqm'] = labelencoder.fit_transform(test_data['ownareaall_sqm'])
test_data['format_name'] = labelencoder.fit_transform(test_data['format_name'])

test_data['startdatetime'] = (test_data['startdatetime'] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')
test_data.drop('group_name', axis=1, inplace=True)

test_data = pd.read_parquet("/home/jupyter/datasphere/project/test_dataset_hackaton2023_test.gzip");
aggregation_functions = {
    'revenue': lambda x: x.median(),  # медиана
    'startdatetime': 'max',  # Последняя дата
    'dish_name': lambda x: x.mode().iloc[0],  # Наиболее часто встречающеес блюд, которые покупал пользователь
    'ownareaall_sqm': lambda x: x.mode().iloc[0],  # Наиболее часто встречающеес площадей ресторана
    'format_name': lambda x: x.mode().iloc[0]    # Наиболее часто встречающееся значение площади ресторана
}

test_data = test_data.groupby(['customer_id']).agg(aggregation_functions).reset_index()

# Применение модели к данным
predicted_values = lgbm_model.predict(test_data.drop('customer_id', axis=1))

# Формирование файла CSV
# Создание нового DataFrame с предсказанными значениями
output_data = test_data[['customer_id']]  # Копирование 'customer_id' и 'startdatetime' из тестовых данных
output_data['buy_post'] = predicted_values

In [ ]:
lgbm_model.predict_proba(test_data.drop('customer_id', axis=1))

In [ ]:
new_order = ['customer_id', 'buy_post', 'revenue', 'startdatetime', 'dish_name', 'ownareaall_sqm', 'format_name']
train_data_reordered = test_data.reindex(columns=new_order)
train_data_reordered['buy_post'] = predicted_values
predicted_values_regress = xgb_model_Regressor.predict(train_data_reordered.drop('customer_id', axis=1))

output_data['startdatetime'] =test_data['startdatetime']
output_data['prob_score'] = lgbm_model.predict_proba(test_data.drop('customer_id', axis=1))[:, 1]


output_data['date_diff_post'] = predicted_values_regress
output_data['date_diff_post'] = output_data['date_diff_post'].astype(int)
output_data.loc[output_data['buy_post'] == 0, 'date_diff_post'] = 0
output_data

In [ ]:
# Сохранение в CSV
output_data.to_csv('predicted_output1.csv', index=False,sep=';')

In [ ]:
total_revenue = test_data['revenue'].sum()
print(total_revenue)

In [ ]:
unique_startdatetimes = test_data['startdatetime'].nunique()
print(unique_startdatetimes)

In [ ]:
orders_per_date_format = test_data.groupby(['startdatetime', 'format_name']).size().reset_index(name='order_count')
orders_per_date_format